In [1]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import pandas as pd
import polars as pl
import polars_st as st
import pyogrio
from electoralyze import region
from electoralyze.common import constants
from electoralyze.common.geometry import to_geopandas, to_geopolars
from electoralyze.common.testing.region_fixture import RegionMocked, read_true_geometry, read_true_metadata
from electoralyze.region.redistribute.mapping import (
    _create_intersection_area_mapping,
    _get_remaining_area,
    create_region_mapping_base,
)

In [2]:
# read_true_geometry("region_a").pipe(to_geopandas).explore()

In [10]:
intersection_area = _create_intersection_area_mapping(
    geometry_from=read_true_geometry("region_b"),
    geometry_to=read_true_geometry("region_c"),
)
intersection_area.pipe(print)

shape: (11, 3)
┌──────────┬──────────┬──────────┐
│ region_b ┆ region_c ┆ mapping  │
│ ---      ┆ ---      ┆ ---      │
│ str      ┆ str      ┆ f64      │
╞══════════╪══════════╪══════════╡
│ A        ┆ X        ┆ 1.21875  │
│ A        ┆ Y        ┆ 0.5      │
│ A        ┆ Z        ┆ 0.28125  │
│ B        ┆ X        ┆ 0.140625 │
│ B        ┆ Y        ┆ 0.25     │
│ …        ┆ …        ┆ …        │
│ C        ┆ X        ┆ 0.140625 │
│ C        ┆ Y        ┆ 0.25     │
│ C        ┆ Z        ┆ 0.609375 │
│ C        ┆ null     ┆ 1.0      │
│ B        ┆ null     ┆ 1.0      │
└──────────┴──────────┴──────────┘


In [11]:
intersection_area.to_dict(as_series=False)

{'region_b': ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'B'],
 'region_c': ['X', 'Y', 'Z', 'X', 'Y', 'Z', 'X', 'Y', 'Z', None, None],
 'mapping': [1.21875,
  0.5,
  0.28125,
  0.140625,
  0.25,
  0.609375,
  0.140625,
  0.25,
  0.609375,
  1.0,
  1.0]}